In [484]:
import pandas as pd
import numpy as np
import math

import time

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm import tqdm

import os

In [485]:
rw = pd.read_excel('./retractions_data/retraction_watch_database.xlsx')
rw.head()

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
0,47271,Binding of DCC by Netrin-1 to Mediate Axon Gui...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Departments of Anatomy and of Biochemistry and...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Yimin Zou;Mu-ming Poo;Marc Tessier-...,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1521,0.0,2001-03-09 00:00:00,10.1126/science.1059391,11239160.0,Retraction,+Investigation by Company/Institution;+Manipul...,No,NaN
1,47270,Hierarchical Organization of Guidance Receptor...,(BLS) Biochemistry;(BLS) Biology - General;(BL...,Department of Anatomy and Department of Bioche...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Marc Tessier-Lavigne,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1517,0.0,2001-02-08 00:00:00,10.1126/science.1058445,11239147.0,Retraction,+Duplication of Image;+Investigation by Compan...,No,NaN
2,47243,Therapeutic potential of targeting IRES-depend...,(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS...,"Division of Hematology-Oncology, UCLA-Greater ...",Oncogene,Springer - Nature Publishing Group,United States,Y Shi;Y Yang;C Bardeleben;B Holmes;J Gera;Alan...,NaN,Research Article;,2023-08-31 00:00:00,10.1038/s41388-023-02820-5,0.0,2015-05-11 00:00:00,10.1038/onc.2015.156,25961916.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://pubpeer.com/publications/704...
3,47233,A classifier based on 273 urinary peptides pre...,(BLS) Biochemistry;(HSC) Medicine - Cardiovasc...,"Department of Nephrology, The Third Affiliated...",Journal of Hypertension,Wolters Kluwer - Lippincott Williams & Wilkins,China,Lirong Lin;Chunxuan Wang;Jiangwen Ren;Mei Mei;...,NaN,Research Article;,2023-08-30 00:00:00,10.1097/HJH.0000000000003551,37642599.0,2023-08-01 00:00:00,10.1097/HJH.0000000000003467,37199562.0,Retraction,+Concerns/Issues About Results;+Investigation ...,No,see also https://journals.lww.com/jhypertensio...
4,47227,"Age, Gender Demographics and Comorbidity Preva...",(HSC) Biostatistics/Epidemiology;(HSC) Medicin...,"Department of Orthopaedics, Dhanalakshmi Srini...",Journal of Coastal Life Medicine,Journal of Coastal Life Medicine,India,S Venkatesh Kumar;Mohith Singh;Gowtham Singh;K...,NaN,Research Article;,2023-08-30 00:00:00,unavailable,0.0,2023-01-01 00:00:00,unavailable,0.0,Retraction,+Notice - Lack of;+Withdrawal;,No,"date of retraction unknown, article title repl..."


In [486]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42700 entries, 0 to 42699
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              42700 non-null  int64  
 1   Title                  42700 non-null  object 
 2   Subject                42700 non-null  object 
 3   Institution            42699 non-null  object 
 4   Journal                42700 non-null  object 
 5   Publisher              42700 non-null  object 
 6   Country                42700 non-null  object 
 7   Author                 42700 non-null  object 
 8   URLS                   21687 non-null  object 
 9   ArticleType            42700 non-null  object 
 10  RetractionDate         42700 non-null  object 
 11  RetractionDOI          42209 non-null  object 
 12  RetractionPubMedID     37599 non-null  float64
 13  OriginalPaperDate      42700 non-null  object 
 14  OriginalPaperDOI       40173 non-null  object 
 15  Or

In [487]:
len(rw['Journal'].unique())

7904

In [488]:
# Group by 'Journal' and check if any group only contains "unavailable" 'OriginalPaperDOI'
journals_with_only_unavailable_doi = rw.groupby('Journal').filter(lambda x: all(x['OriginalPaperDOI'].str.lower() == 'unavailable'))

# Extract unique journal names from the result
journals_list = journals_with_only_unavailable_doi['Journal'].unique()
len(journals_list)

1054

In [489]:
sum(rw['Journal'].isin(journals_list))

2442

In [490]:
rw[rw['Journal'].isin(journals_list)]['OriginalPaperDOI'].value_counts()

OriginalPaperDOI
Unavailable    1867
unavailable     575
Name: count, dtype: int64

In [491]:
len(rw[rw['OriginalPaperDOI'] == 'unavailable']) + len(rw[rw['OriginalPaperDOI'] == 'Unavailable'])

3308

In [492]:
len(rw[rw['Journal'].isin(journals_list)])

2442

In [493]:
aux_rw = rw.copy()
aux_rw['OriginalPaperDOI'] = aux_rw['OriginalPaperDOI'].str.lower()
aux_rw = aux_rw[aux_rw['OriginalPaperDOI'] != 'unavailable']
aux_rw.dropna(subset=['OriginalPaperDOI'], inplace=True)
first_doi_per_journal = aux_rw.groupby('Journal')['OriginalPaperDOI'].first()
first_doi_per_journal

Journal
'Herbst des Mittelalters' Fragen zur Bewertung des 14th und 15th Jahrhunderts                                       10.1515/9783110204544.6.277
... fÃ¼r unseren Betrieb lebensnotwendig ...: Georg von Holtzbrinck als Verlagsunternehmer im Dritten Reich           10.1515/9783598441240.toc
10th IEEE International Conference on Electronics, Circuits and Systems, 2003. ICECS 2003                            10.1109/icecs.2003.1302059
10th International Conference on Telecommunications, 2003                                                            10.1109/ictel.2003.1191486
12th IEEE International Conference and Workshops on the Engineering of Computer-Based Systems (ECBS'05)                    10.1109/ecbs.2005.74
                                                                                                                             ...               
medRxiv                                                                                                             10.1101/2021

In [494]:
aux_rw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36865 entries, 0 to 42697
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              36865 non-null  int64  
 1   Title                  36865 non-null  object 
 2   Subject                36865 non-null  object 
 3   Institution            36864 non-null  object 
 4   Journal                36865 non-null  object 
 5   Publisher              36865 non-null  object 
 6   Country                36865 non-null  object 
 7   Author                 36865 non-null  object 
 8   URLS                   19043 non-null  object 
 9   ArticleType            36865 non-null  object 
 10  RetractionDate         36865 non-null  object 
 11  RetractionDOI          36623 non-null  object 
 12  RetractionPubMedID     34032 non-null  float64
 13  OriginalPaperDate      36865 non-null  object 
 14  OriginalPaperDOI       36865 non-null  object 
 15  Origina

In [495]:
aux_rw.groupby('Journal')['OriginalPaperDOI'].nth(1)

1                   10.1126/science.1058445
22                        10.1002/brb3.2552
24                        10.1002/hsr2.1068
46         10.1097/00003246-199407000-00018
48                10.1007/s10586-017-1440-0
                        ...                
42549               10.1176/ajp.139.11.1447
42582    10.1016/b978-0-12-484772-9.50007-4
42583    10.1016/b978-0-12-484760-6.50007-2
42625          10.1016/0002-9149(80)91075-9
42697                    10.1007/bf01488952
Name: OriginalPaperDOI, Length: 2995, dtype: object

In [694]:
articles_per_journal = aux_rw.groupby('Journal').size()
articles_per_journal

Journal
'Herbst des Mittelalters' Fragen zur Bewertung des 14th und 15th Jahrhunderts                                   1
... fÃ¼r unseren Betrieb lebensnotwendig ...: Georg von Holtzbrinck als Verlagsunternehmer im Dritten Reich     3
10th IEEE International Conference on Electronics, Circuits and Systems, 2003. ICECS 2003                       1
10th International Conference on Telecommunications, 2003                                                       1
12th IEEE International Conference and Workshops on the Engineering of Computer-Based Systems (ECBS'05)         1
                                                                                                               ..
medRxiv                                                                                                        22
medrxiv                                                                                                         1
microPublication biology                                                        

In [693]:
#articles_per_journal = aux_rw.groupby('Journal').ngroup()
#articles_per_journal

0        5869
1        5869
2        5122
3        3876
5        5899
         ... 
42692    4941
42693    2895
42694    6259
42696    1925
42697    1925
Length: 36865, dtype: int64

In [497]:
articles_per_journal[articles_per_journal>1]

Journal
... fÃ¼r unseren Betrieb lebensnotwendig ...: Georg von Holtzbrinck als Verlagsunternehmer im Dritten Reich     3
2004 Symposium on VLSI Circuits                                                                                 2
2005 IEEE Radio Frequency integrated Circuits (RFIC) Symposium                                                  3
2005 Symposium on VLSI Circuits                                                                                 2
2006 Bipolar/BiCMOS Circuits and Technology Meeting                                                             3
                                                                                                               ..
eLife                                                                                                           9
iScience                                                                                                        3
mBio                                                                            

In [498]:
articles_per_journal.describe()

count    6710.000000
mean        5.494039
std        31.447845
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max      1240.000000
dtype: float64

In [499]:
articles_per_journal.sort_values(inplace= False)[-50:]

Journal
Cancer Management and Research                                                                                        101
2010 2nd International Conference on Computer Engineering and Technology                                              105
Science                                                                                                               107
Cellular Physiology and Biochemistry                                                                                  108
Cluster Computing                                                                                                     116
Journal of Clinical Anesthesia                                                                                        117
Multimedia Tools and Applications                                                                                     122
Journal of Cellular Physiology                                                                                        123
International Jo

In [500]:
first_doi_per_journal[0:15].values

array(['10.1515/9783110204544.6.277', '10.1515/9783598441240.toc',
       '10.1109/icecs.2003.1302059', '10.1109/ictel.2003.1191486',
       '10.1109/ecbs.2005.74', '10.1109/iciapw.2007.24',
       '10.1109/adcom.2007.13', '10.1109/coniel.2005.68',
       '10.22616/erdev2019.18.n232', '10.1109/aswec.2008.4483193',
       '10.1109/aero.2001.931370', '10.1109/isscc.2004.1332736',
       '10.1109/vlsic.2004.1346631', '10.1109/rfic.2005.1489795',
       '10.1109/esscir.2005.1541567'], dtype=object)

In [501]:
rw[rw['OriginalPaperDOI']=="None"]

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes


In [502]:
sum(first_doi_per_journal=="None")

0

In [503]:
rw['ArticleType'].unique()

array(['Research Article;', 'Clinical Study;', 'Commentary/Editorial;',
       'Review Article;', 'Meta-Analysis;', 'Letter;Research Article;',
       'Case Report;', 'Commentary/Editorial;Review Article;',
       'Article in Press;Preprint;Research Article;',
       'Conference Abstract/Paper;', 'Meta-Analysis;Review Article;',
       'Preprint;Research Article;', 'Preprint;Review Article;',
       'Clinical Study;Research Article;',
       'Research Article;Review Article;',
       'Article in Press;Research Article;',
       'Article in Press;Review Article;', 'Commentary/Editorial;Letter;',
       'Conference Abstract/Paper;Review Article;',
       'Dissertation/Thesis;', 'Letter;', 'Retraction Notice;',
       'Clinical Study;Letter;',
       'Meta-Analysis;Research Article;Review Article;',
       'Meta-Analysis;Research Article;', 'Letter;Review Article;',
       'Case Report;Letter;', 'Retracted Article;',
       'Letter;Meta-Analysis;',
       'Article in Press;Conference Abst

#### Filtered to contain only articles

In [504]:
article_type = rw[['Record ID', 'ArticleType']].copy()
article_type.loc[:, 'ArticleType'] = article_type['ArticleType'].str.split(';')
article_type = article_type.explode('ArticleType')
article_type = article_type[article_type['ArticleType'].str.strip() != ""]
article_type = article_type[article_type['ArticleType'].str.strip() != ";"]
article_type = article_type['ArticleType'].str.strip()
article_type

0                    Research Article
1                    Research Article
2                    Research Article
3                    Research Article
4                    Research Article
                     ...             
42695                Research Article
42696                  Review Article
42697                  Review Article
42698                  Review Article
42699    Technical Report/White Paper
Name: ArticleType, Length: 44066, dtype: object

In [505]:
article_type.unique()

array(['Research Article', 'Clinical Study', 'Commentary/Editorial',
       'Review Article', 'Meta-Analysis', 'Letter', 'Case Report',
       'Article in Press', 'Preprint', 'Conference Abstract/Paper',
       'Dissertation/Thesis', 'Retraction Notice', 'Retracted Article',
       'Guideline', 'Other', 'Trade Magazines',
       'Technical Report/White Paper', 'Correction/Erratum/Corrigendum',
       'Book Chapter/Reference Work', 'Auto/Biography', 'Revision',
       'Supplementary Materials', 'Government Publication',
       'Expression of Concern', 'Interview/Q&A', 'Legal Case/Analysis'],
      dtype=object)

In [506]:
select_article_type = ['Research Article', 'Clinical Study', 'Commentary/Editorial',
       'Review Article', 'Meta-Analysis', 'Case Report',  
       'Retraction Notice', 'Retracted Article',
       'Correction/Erratum/Corrigendum',
       'Supplementary Materials','Legal Case/Analysis'] 
filtered_df = rw[rw['ArticleType'].str.contains('|'.join(select_article_type))]

In [507]:
filtered_df

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
0,47271,Binding of DCC by Netrin-1 to Mediate Axon Gui...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Departments of Anatomy and of Biochemistry and...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Yimin Zou;Mu-ming Poo;Marc Tessier-...,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1521,0.0,2001-03-09 00:00:00,10.1126/science.1059391,11239160.0,Retraction,+Investigation by Company/Institution;+Manipul...,No,NaN
1,47270,Hierarchical Organization of Guidance Receptor...,(BLS) Biochemistry;(BLS) Biology - General;(BL...,Department of Anatomy and Department of Bioche...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Marc Tessier-Lavigne,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,2023-08-31 00:00:00,10.1126/science.adk1517,0.0,2001-02-08 00:00:00,10.1126/science.1058445,11239147.0,Retraction,+Duplication of Image;+Investigation by Compan...,No,NaN
2,47243,Therapeutic potential of targeting IRES-depend...,(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS...,"Division of Hematology-Oncology, UCLA-Greater ...",Oncogene,Springer - Nature Publishing Group,United States,Y Shi;Y Yang;C Bardeleben;B Holmes;J Gera;Alan...,NaN,Research Article;,2023-08-31 00:00:00,10.1038/s41388-023-02820-5,0.0,2015-05-11 00:00:00,10.1038/onc.2015.156,25961916.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://pubpeer.com/publications/704...
3,47233,A classifier based on 273 urinary peptides pre...,(BLS) Biochemistry;(HSC) Medicine - Cardiovasc...,"Department of Nephrology, The Third Affiliated...",Journal of Hypertension,Wolters Kluwer - Lippincott Williams & Wilkins,China,Lirong Lin;Chunxuan Wang;Jiangwen Ren;Mei Mei;...,NaN,Research Article;,2023-08-30 00:00:00,10.1097/HJH.0000000000003551,37642599.0,2023-08-01 00:00:00,10.1097/HJH.0000000000003467,37199562.0,Retraction,+Concerns/Issues About Results;+Investigation ...,No,see also https://journals.lww.com/jhypertensio...
4,47227,"Age, Gender Demographics and Comorbidity Preva...",(HSC) Biostatistics/Epidemiology;(HSC) Medicin...,"Department of Orthopaedics, Dhanalakshmi Srini...",Journal of Coastal Life Medicine,Journal of Coastal Life Medicine,India,S Venkatesh Kumar;Mohith Singh;Gowtham Singh;K...,NaN,Research Article;,2023-08-30 00:00:00,unavailable,0.0,2023-01-01 00:00:00,unavailable,0.0,Retraction,+Notice - Lack of;+Withdrawal;,No,"date of retraction unknown, article title repl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42694,825,On the Primary Site of Nuclear RNA Synthesis,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Department of Biology University of Pennsylvan...,The Journal of Cell Biology,Rockefeller University Press,United States,Louise M Luckenbill;Alan S Cohen,http://retractionwatch.com/2013/04/24/authors-...,Research Article;,1966-10-01 00:00:00,10.1083/jcb.31.1.195,5971970.0,1959-10-01 00:00:00,10.1083/jcb.6.2.301,13850774.0,Retraction,+Error in Results and/or Conclusions;+Results ...,No,NaN
42695,10433,Change of Venue and the Conflict of Laws,(SOC) Law/Legal Issues;,University of Chicago;,The University of Chicago Law Review,University of Chicago Law School,United States,Brainerd Currie,NaN,Research Article;,1960-02-01 00:00:00,NaN,NaN,1955-02-01 00:00:00,NaN,NaN,Retraction,+Error in Results and/or Conclusions;+Unreliab...,Yes,NaN
42696,18529,Sinn und Grenzen der exakten Wissenschaft,(PHY) Physics;,Berlin,Die Naturwissenschaften,Springer,Germany,Max Planck,NaN,Review Article;,1942-02-01 00:00:00,10.1007/BF01475382,0.0,1942-02-01 00:00:00,10.1007/BF01475382,0.0,Retraction,+Copyright Claims;,No,NaN
42697,18531,Naturwissens

In [508]:
filtered_df['Journal'].value_counts()[:50]

Journal
PLoS One                                                                                                             642
Cochrane Database of Systematic Reviews                                                                              574
Journal of Ambient Intelligence and Humanized Computing                                                              430
The Journal of Biological Chemistry                                                                                  379
Arabian Journal of Geosciences                                                                                       368
Evidence-Based Complementary and Alternative Medicine (Ecam)                                                         334
European Review for Medical and Pharmacological Sciences                                                             297
Journal of Healthcare Engineering                                                                                    271
Computational Intelligen

In [509]:
len(filtered_df['Journal'].unique())

6854

In [510]:
filtered_df['OriginalPaperDOI']

0             10.1126/science.1059391
1             10.1126/science.1058445
2                10.1038/onc.2015.156
3        10.1097/HJH.0000000000003467
4                         unavailable
                     ...             
42694             10.1083/jcb.6.2.301
42695                             NaN
42696              10.1007/BF01475382
42697              10.1007/BF01488952
42698                     unavailable
Name: OriginalPaperDOI, Length: 29401, dtype: object

aux_rw = filtered_df.copy()
aux_rw['OriginalPaperDOI'] = aux_rw['OriginalPaperDOI'].str.lower()
aux_rw = aux_rw[aux_rw['OriginalPaperDOI'] != 'unavailable']
first_doi_per_journal = aux_rw.groupby('Journal')['OriginalPaperDOI'].last()
first_doi_per_journal

### Scrape the DOIs

In [25]:
start_doi = 6440
end_doi = len(first_doi_per_journal)
website_path = "https://www.webofscience.com/wos/woscc/summary/3e49766e-1c18-4edf-8d19-1ab46e85eb71-ccdba0d9/relevance/1"

In [26]:
# path for chrome driver
path = Service(ChromeDriverManager().install())

# create the driver
driver = webdriver.Chrome(service = path)

# open chromedriver window
driver.get(website_path)
time.sleep(10)

driver.maximize_window()
time.sleep(3)

# reject cookies
driver.find_element('xpath', '//*[@id="onetrust-reject-all-handler"]').click()

In [27]:
from selenium.common.exceptions import ElementClickInterceptedException

In [28]:
#dois= first_doi_per_journal[start_doi:end_doi].values

exception = False

for first_doi_range in tqdm(range(start_doi, end_doi, 20)):
    try: 
        # click on search bar
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()
    except:
        time.sleep(30)
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

    if exception: 
        # click on search bar
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

    try: 
        # clear query box
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()
    except:
        time.sleep(3)
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()

    # select 10 DOIs at a time and create query
    time.sleep(0.5)
    selected_dois = first_doi_per_journal[first_doi_range:first_doi_range+20]
    query = 'DO=("' + '" OR "'.join(map(str, selected_dois)) + '")'
    driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(query)

    # click search 
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div/div[4]/div[2]/app-advanced-search-form/form/div[2]/div[1]/div/div/button[2]/span[1]').click()
    time.sleep(3)

    # calculate total number of publications
    n_publications = int(driver.find_element(By.CLASS_NAME, "brand-blue").text.replace(',', ''))

    # open "Export" options
    try: 
        driver.find_element('xpath', '//*[@id="snRecListTop"]/app-export-menu/div/button').click()
        exception = False
        time.sleep(2)
    except ElementClickInterceptedException:
        print("The DOIs from range ", first_doi_range, " to ", first_doi_range+20, " are not available")
        exception = True
        time.sleep(5)
        continue

    # export to excel file
    driver.find_element('xpath', '//*[@id="exportToExcelButton"]').click()

    #default is export all records on page and content that includes issn. 
    # click download button
    time.sleep(2)
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]/span[1]/span').click()
    time.sleep(8)

  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [04:02<00:00, 17.33s/it]


In [24]:
first_doi_range

6440

In [22]:
selected_dois

array(['10.1109/indusis.2010.5565783', '10.1109/iciecs.2010.5678419',
       '10.1109/itcs.2010.63', '10.1109/icmee.2010.5558447',
       '10.1109/nswctc.2010.133', '10.1109/icacte.2010.5579258',
       '10.1109/bmei.2010.5639857', '10.1109/iccsit.2010.5564645',
       '10.1109/icetet.2010.140', '10.1109/iciii.2010.588'], dtype=object)

In [24]:
first_doi_per_journal[130:140].values

array(['10.1109/indusis.2010.5565783', '10.1109/iciecs.2010.5678419',
       '10.1109/itcs.2010.63', '10.1109/icmee.2010.5558447',
       '10.1109/nswctc.2010.133', '10.1109/icacte.2010.5579258',
       '10.1109/bmei.2010.5639857', '10.1109/iccsit.2010.5564645',
       '10.1109/icetet.2010.140', '10.1109/iciii.2010.588'], dtype=object)

### Second Iteration of Scraping the DOIs

In [661]:
issn_files = os.listdir('../thesis_data/journal_issns')

In [662]:
issn_articles = pd.DataFrame()
for folder in os.listdir('../thesis_data/journal_issns'): 
    for file in os.listdir('../thesis_data/journal_issns/' + folder):
        aux = pd.read_excel('../thesis_data/journal_issns/' + folder + '/' + file)
        issn_articles = pd.concat([issn_articles, aux], ignore_index = True)

In [663]:
issn_articles.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa

In [664]:
rw_w_issns = aux_rw.merge(issn_articles[['DOI', 'ISSN']], left_on = "OriginalPaperDOI", right_on = 'DOI', how= 'left')
rw_w_issns

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,...,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes,DOI,ISSN
0,47271,Binding of DCC by Netrin-1 to Mediate Axon Gui...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Departments of Anatomy and of Biochemistry and...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Yimin Zou;Mu-ming Poo;Marc Tessier-...,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,...,0.0,2001-03-09 00:00:00,10.1126/science.1059391,11239160.0,Retraction,+Investigation by Company/Institution;+Manipul...,No,NaN,10.1126/science.1059391,0036-8075
1,47270,Hierarchical Organization of Guidance Receptor...,(BLS) Biochemistry;(BLS) Biology - General;(BL...,Department of Anatomy and Department of Bioche...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Marc Tessier-Lavigne,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,...,0.0,2001-02-08 00:00:00,10.1126/science.1058445,11239147.0,Retraction,+Duplication of Image;+Investigation by Compan...,No,NaN,NaN,NaN
2,47243,Therapeutic potential of targeting IRES-depend...,(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS...,"Division of Hematology-Oncology, UCLA-Greater ...",Oncogene,Springer - Nature Publishing Group,United States,Y Shi;Y Yang;C Bardeleben;B Holmes;J Gera;Alan...,NaN,Research Article;,...,0.0,2015-05-11 00:00:00,10.1038/onc.2015.156,25961916.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://pubpeer.com/publications/704...,10.1038/onc.2015.156,0950-9232
3,47233,A classifier based on 273 urinary peptides pre...,(BLS) Biochemistry;(HSC) Medicine - Cardiovasc...,"Department of Nephrology, The Third Affiliated...",Journal of Hypertension,Wolters Kluwer - Lippincott Williams & Wilkins,China,Lirong Lin;Chunxuan Wang;Jiangwen Ren;Mei Mei;...,NaN,Research Article;,...,37642599.0,2023-08-01 00:00:00,10.1097/hjh.0000000000003467,37199562.0,Retraction,+Concerns/Issues About Results;+Investigation ...,No,see also https://journals.lww.com/jhypertensio...,NaN,NaN
4,47230,"The Hopewell airburst event, 1699â€“1567 years...",(BLS) Anthropology;(BLS) Archeology;(PHY) Cosm...,"Department of Anthropology, Department of Geol...",Scientific Reports,Springer - Nature Publishing Group,United States,Kenneth Barnett Tankersley;Stephen D Meyers;St...,NaN,Research Article;,...,37648734.0,2022-02-01 00:00:00,10.1038/s41598-022-05758-y,35105925.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://www.nature.com/articles/s415...,10.1038/s41598-022-05758-y,2045-2322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36942,44154,Effect of Host Pregnancy on Pupal Production b...,(BLS) Zoology;(HSC) Veterinary Science;,"Tsetse Research Laboratory, University of Bris...",Nature,Springer - Nature Publishing Group,United Kingdom,T A M Nash;A M Jordan;J A Boyle,NaN,Research Article;,...,0.0,1966-12-01 00:00:00,10.1038/2121581b0,21108521.0,Retraction,+Error in Results and/or Conclusions;+Results ...,No,NaN,NaN,NaN
36943,17192,Paramagnetic Relaxation in Inorganic Complexes...,(PHY) Chemistry;(PHY) Physics;,"Department of Chemistry, Michigan State Univer...",Inorganica Chimica Acta,Elsevier,United States,L A Dalton;L L Dalton;Larry R Dalton,http://retractionwatch.com/2014/02/25/leading-...,Research Article;,...,0.0,1967-06-01 00:00:00,10.1016/s0020-1693(00)93130-x,0.0,Retraction,+Falsification/Fabrication of Data;+Plagiarism...,No,NaN,NaN,NaN
36944,825,On the Primary Site of Nuclear RNA Synthesis,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Department of Biology University of Pennsylvan...,The Journal of Cell Biology,Rockefeller University Press,United States,Louise M Luckenbill;Alan S Cohen,http://retractionwatch.com/2013/04/24/authors-...,Research Article;,...,5971970.0,1959-10-01 00

In [665]:
rw_w_issns.drop_duplicates(inplace=True)
rw_w_issns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36868 entries, 0 to 36946
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              36868 non-null  int64  
 1   Title                  36868 non-null  object 
 2   Subject                36868 non-null  object 
 3   Institution            36867 non-null  object 
 4   Journal                36868 non-null  object 
 5   Publisher              36868 non-null  object 
 6   Country                36868 non-null  object 
 7   Author                 36868 non-null  object 
 8   URLS                   19043 non-null  object 
 9   ArticleType            36868 non-null  object 
 10  RetractionDate         36868 non-null  object 
 11  RetractionDOI          36626 non-null  object 
 12  RetractionPubMedID     34035 non-null  float64
 13  OriginalPaperDate      36868 non-null  object 
 14  OriginalPaperDOI       36868 non-null  object 
 15  Origina

In [666]:
# journals without ISSN
journal_counts = rw_w_issns.groupby('Journal')['ISSN'].count()
journal_counts[journal_counts == 0].index.to_list()

["'Herbst des Mittelalters' Fragen zur Bewertung des 14th und 15th Jahrhunderts",
 '... fÃ¼r unseren Betrieb lebensnotwendig ...: Georg von Holtzbrinck als Verlagsunternehmer im Dritten Reich',
 '10th IEEE International Conference on Electronics, Circuits and Systems, 2003. ICECS 2003',
 '10th International Conference on Telecommunications, 2003',
 "12th IEEE International Conference and Workshops on the Engineering of Computer-Based Systems (ECBS'05)",
 '14th International Conference of Image Analysis and Processing - Workshops',
 '15th International Conference on Advanced Computing and Communications (ADCOM 2007)',
 "15th International Conference on Electronics, Communications and Computers (CONIELECOMP'05)",
 '18th International Scientific Conference Engineering for Rural Development',
 '19th Australian Conference on Software Engineering (aswec 2008)',
 '2001 IEEE Aerospace Conference Proceeding',
 '2004 IEEE International Solid-State Circuits Conference',
 '2004 Symposium on VLSI C

In [667]:
# journals that only have 1 article
articles_per_journal[articles_per_journal==1]

Journal
'Herbst des Mittelalters' Fragen zur Bewertung des 14th und 15th Jahrhunderts                              1
10th IEEE International Conference on Electronics, Circuits and Systems, 2003. ICECS 2003                  1
10th International Conference on Telecommunications, 2003                                                  1
12th IEEE International Conference and Workshops on the Engineering of Computer-Based Systems (ECBS'05)    1
14th International Conference of Image Analysis and Processing - Workshops                                 1
                                                                                                          ..
mSphere                                                                                                    1
mUX: The Journal of Mobile User Experience                                                                 1
medrxiv                                                                                                    1
npj Digital

In [668]:
# journals with more than 1 article
articles_per_journal[articles_per_journal>1]

Journal
... fÃ¼r unseren Betrieb lebensnotwendig ...: Georg von Holtzbrinck als Verlagsunternehmer im Dritten Reich     3
2004 Symposium on VLSI Circuits                                                                                 2
2005 IEEE Radio Frequency integrated Circuits (RFIC) Symposium                                                  3
2005 Symposium on VLSI Circuits                                                                                 2
2006 Bipolar/BiCMOS Circuits and Technology Meeting                                                             3
                                                                                                               ..
eLife                                                                                                           9
iScience                                                                                                        3
mBio                                                                            

In [669]:
# list of journals that do not have ISSNs but have more than one article 
journals_w_more_articles = list(set(journal_counts[journal_counts == 0].index.to_list()).intersection(articles_per_journal[articles_per_journal>1].index.to_list()))
journals_w_more_articles

['Modern Physics Letters B',
 'Journal of Applied Pharmaceutical Science',
 'Counterterrorism and Open Source Intelligence',
 'Economy, Statistics and Informatics. Vestnik Umo (Ð\xadÐºÐ¾Ð½Ð¾Ð¼Ð¸ÐºÐ°, Ð¡Ñ‚Ð°Ñ‚Ð¸Ñ\x81Ñ‚Ð¸ÐºÐ° Ð˜ Ð˜Ð½Ñ„Ð¾Ñ€Ð¼Ð°Ñ‚Ð¸ÐºÐ°. Ð’ÐµÑ\x81Ñ‚Ð½Ð¸Ðº Ð£Ð¼Ð¾)',
 'Circulation Research',
 'Neurological Sciences and Neurophysiology',
 'Nutrition Clinique et MÃ©tabolisme',
 'Water Resources Research',
 '2009 International Conference on Information Engineering and Computer Science',
 '2005 IEEE Radio Frequency integrated Circuits (RFIC) Symposium',
 'Australasian Medical Journal',
 'Food and Nutrition Sciences',
 'Der Ã¶sterreichische StÃ¤ndestaat 1934â€“1938',
 'Molecules',
 '2009 International Conference on Computer Engineering and Technology',
 '2010 International Conference on Logistics Engineering and Intelligent Transportation Systems',
 'Russian Linguistic Bulletin',
 'Lancet',
 'The Journal of Bone and Joint Surgery',
 '2009 Asia-Pacific Conference on Information Pr

In [670]:
len(journals_w_more_articles)

914

In [671]:
len(set(journal_counts[journal_counts == 0].index.to_list()).intersection(articles_per_journal[articles_per_journal==1].index.to_list()))

2375

In [686]:
aux_rw = rw.copy()
aux_rw['OriginalPaperDOI'] = aux_rw['OriginalPaperDOI'].str.lower()
aux_rw = aux_rw[aux_rw['OriginalPaperDOI'] != 'unavailable']
aux_rw.dropna(subset=['OriginalPaperDOI'], inplace=True)
doi_per_journal = aux_rw.groupby('Journal', dropna=False)[['OriginalPaperDOI', 'Record ID']].nth(2) 
doi_per_journal

,OriginalPaperDOI,Record ID
49,10.1097/00003246-199701000-00019,47030
59,10.1080/13625187.2020.1868427,47012
93,10.2147/cmar.s208773,46976
94,10.1002/bab.1310,46824
101,10.2147/ott.s238070,46975
...,...,...
42584,10.1016/b978-0-12-484772-9.50011-6,41273
42585,10.1016/b978-0-12-484760-6.50011-4,41276
42626,10.1016/0002-9149(80)90690-6,19404
42650,10.1016/0002-9343(80)90167-9,1132


In [687]:
doi_per_journal = doi_per_journal.merge(rw[['Record ID', 'Journal']], how='left', on = 'Record ID')
doi_per_journal.set_index('Journal', inplace = True)
doi_per_journal.drop(['Record ID'], axis = 1, inplace = True)
doi_per_journal = doi_per_journal.squeeze()
doi_per_journal

Journal
Critical Care Medicine                                                10.1097/00003246-199701000-00019
European Journal of Contraception & Reproductive Health Care             10.1080/13625187.2020.1868427
Cancer Management and Research                                                    10.2147/cmar.s208773
Biotechnology and Applied Biochemistry                                                10.1002/bab.1310
OncoTargets and Therapy                                                            10.2147/ott.s238070
                                                                                   ...                
Intermediate Algebra, 3rd ed                                        10.1016/b978-0-12-484772-9.50011-6
Intermediate Algebra                                                10.1016/b978-0-12-484760-6.50011-4
The American Journal of Cardiology                                        10.1016/0002-9149(80)90690-6
The American Journal of Medicine                                 

In [688]:
doi_per_journal = selected_rows = doi_per_journal[doi_per_journal.index.isin(journals_w_more_articles)]
doi_per_journal

Journal
Cancer Management and Research                                                                      10.2147/cmar.s208773
OncoTargets and Therapy                                                                              10.2147/ott.s238070
Heliyon                                                                                    10.1016/j.heliyon.2022.e12031
IOP Conference Series: Materials Science and Engineering                                 10.1088/1757-899x/1049/1/012013
Biomedical Materials                                                                            10.1088/1748-605x/ac08aa
                                                                                                     ...                
Wolfram Dorn, Willi Weiskirch in Abgeordnete des Deutschen Bundestages Book Series             10.1515/9783486830262.toc
Linear Programming: An Introduction with Applications                                 10.1016/b978-0-12-676350-8.50015-3
Intermediate Algebra, 3r

In [624]:
start_doi = 0
end_doi = len(doi_per_journal)
website_path = "https://www.webofscience.com/wos/woscc/summary/3e49766e-1c18-4edf-8d19-1ab46e85eb71-ccdba0d9/relevance/1"

In [625]:
# path for chrome driver
path = Service(ChromeDriverManager().install())

# create the driver
driver = webdriver.Chrome(service = path)

# open chromedriver window
driver.get(website_path)
time.sleep(10)

driver.maximize_window()
time.sleep(3)

# reject cookies
driver.find_element('xpath', '//*[@id="onetrust-reject-all-handler"]').click()

In [626]:
from selenium.common.exceptions import ElementClickInterceptedException

In [627]:
#dois= doi_per_journal[start_doi:end_doi].values

exception = False

for first_doi_range in tqdm(range(start_doi, end_doi, 20)):
    try: 
        # click on search bar
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()
    except:
        time.sleep(30)
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

    if exception: 
        # click on search bar
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div').click()

    try: 
        # clear query box
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()
    except:
        time.sleep(3)
        driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').clear()

    # select 10 DOIs at a time and create query
    time.sleep(0.5)
    selected_dois = doi_per_journal[first_doi_range:first_doi_range+20]
    query = 'DO=("' + '" OR "'.join(map(str, selected_dois)) + '")'
    driver.find_element('xpath','//*[@id="advancedSearchInputArea"]').send_keys(query)

    # click search 
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/app-query-modifier/div/div[1]/div/div[4]/div[2]/app-advanced-search-form/form/div[2]/div[1]/div/div/button[2]/span[1]').click()
    time.sleep(3)

    # calculate total number of publications
    n_publications = int(driver.find_element(By.CLASS_NAME, "brand-blue").text.replace(',', ''))

    # open "Export" options
    try: 
        driver.find_element('xpath', '//*[@id="snRecListTop"]/app-export-menu/div/button').click()
        exception = False
        time.sleep(2)
    except ElementClickInterceptedException:
        print("The DOIs from range ", first_doi_range, " to ", first_doi_range+20, " are not available")
        exception = True
        time.sleep(5)
        continue

    # export to plaintext file
    driver.find_element('xpath', '//*[@id="exportToFieldTaggedButton"]').click()
    

    #default is export all records on page and content that includes issn. 
    time.sleep(2)

    # dropdown "Record Content:"
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button').click()

    time.sleep(2)
    
    # choose the record content to be "Full Record and Cited References"
    driver.find_element('xpath', '//*[@id="global-select"]/div/div/div[4]/span').click()

    time.sleep(2)
    
    # click export
    driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]').click()

    time.sleep(13)

 50%|█████     | 3/6 [00:50<00:50, 16.76s/it]

The DOIs from range  60  to  80  are not available


100%|██████████| 6/6 [01:33<00:00, 15.66s/it]


In [594]:
first_doi_range

100

In [595]:
selected_dois

Journal
2009 International Conference on Computer Technology and Development                                         10.1109/icctd.2009.36
2009 International Conference on Information Engineering and Computer Science                          10.1109/iciecs.2009.5364392
2009 International Conference on Computational Intelligence and Software Engineering                     10.1109/cise.2009.5365291
2009 International Conference on E-Learning, E-Business, Enterprise Information Sytms & E-Gvmt                10.1109/eeee.2009.56
2009 International Conference on Management and Service Science                                         10.1109/icmss.2009.5303114
Lancet                                                                                               10.1016/s0140-6736(03)12229-5
2009 2nd IEEE International Conference on Computer Science and Information Technology                  10.1109/iccsit.2009.5234715
2009 International Forum on Information Technology and Applications        

# Merge into rw and analyse

In [641]:
issn_files = os.listdir('../thesis_data/journal_issns')

In [642]:
issn_articles = pd.DataFrame()
for folder in os.listdir('../thesis_data/journal_issns'): 
    for file in os.listdir('../thesis_data/journal_issns/' + folder):
        aux = pd.read_excel('../thesis_data/journal_issns/' + folder + '/' + file)
        issn_articles = pd.concat([issn_articles, aux], ignore_index = True)

In [644]:
issn_articles.shape

(6384, 72)

In [645]:
rw_w_issns = rw.merge(issn_articles[['DOI', 'ISSN']], left_on = "OriginalPaperDOI", right_on = 'DOI', how= 'left')
rw_w_issns

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,...,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes,DOI,ISSN
0,47271,Binding of DCC by Netrin-1 to Mediate Axon Gui...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,Departments of Anatomy and of Biochemistry and...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Yimin Zou;Mu-ming Poo;Marc Tessier-...,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,...,0.0,2001-03-09 00:00:00,10.1126/science.1059391,11239160.0,Retraction,+Investigation by Company/Institution;+Manipul...,No,NaN,10.1126/science.1059391,0036-8075
1,47270,Hierarchical Organization of Guidance Receptor...,(BLS) Biochemistry;(BLS) Biology - General;(BL...,Department of Anatomy and Department of Bioche...,Science,American Association for the Advancement of Sc...,United States,Elke Stein;Marc Tessier-Lavigne,https://retractionwatch.com/2023/08/31/stanfor...,Research Article;,...,0.0,2001-02-08 00:00:00,10.1126/science.1058445,11239147.0,Retraction,+Duplication of Image;+Investigation by Compan...,No,NaN,NaN,NaN
2,47243,Therapeutic potential of targeting IRES-depend...,(BLS) Biochemistry;(BLS) Biology - Cancer;(BLS...,"Division of Hematology-Oncology, UCLA-Greater ...",Oncogene,Springer - Nature Publishing Group,United States,Y Shi;Y Yang;C Bardeleben;B Holmes;J Gera;Alan...,NaN,Research Article;,...,0.0,2015-05-11 00:00:00,10.1038/onc.2015.156,25961916.0,Retraction,+Concerns/Issues About Data;+Concerns/Issues A...,No,see also: https://pubpeer.com/publications/704...,10.1038/onc.2015.156,0950-9232
3,47233,A classifier based on 273 urinary peptides pre...,(BLS) Biochemistry;(HSC) Medicine - Cardiovasc...,"Department of Nephrology, The Third Affiliated...",Journal of Hypertension,Wolters Kluwer - Lippincott Williams & Wilkins,China,Lirong Lin;Chunxuan Wang;Jiangwen Ren;Mei Mei;...,NaN,Research Article;,...,37642599.0,2023-08-01 00:00:00,10.1097/HJH.0000000000003467,37199562.0,Retraction,+Concerns/Issues About Results;+Investigation ...,No,see also https://journals.lww.com/jhypertensio...,NaN,NaN
4,47227,"Age, Gender Demographics and Comorbidity Preva...",(HSC) Biostatistics/Epidemiology;(HSC) Medicin...,"Department of Orthopaedics, Dhanalakshmi Srini...",Journal of Coastal Life Medicine,Journal of Coastal Life Medicine,India,S Venkatesh Kumar;Mohith Singh;Gowtham Singh;K...,NaN,Research Article;,...,0.0,2023-01-01 00:00:00,unavailable,0.0,Retraction,+Notice - Lack of;+Withdrawal;,No,"date of retraction unknown, article title repl...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42909,10433,Change of Venue and the Conflict of Laws,(SOC) Law/Legal Issues;,University of Chicago;,The University of Chicago Law Review,University of Chicago Law School,United States,Brainerd Currie,NaN,Research Article;,...,NaN,1955-02-01 00:00:00,NaN,NaN,Retraction,+Error in Results and/or Conclusions;+Unreliab...,Yes,NaN,NaN,NaN
42910,18529,Sinn und Grenzen der exakten Wissenschaft,(PHY) Physics;,Berlin,Die Naturwissenschaften,Springer,Germany,Max Planck,NaN,Review Article;,...,0.0,1942-02-01 00:00:00,10.1007/BF01475382,0.0,Retraction,+Copyright Claims;,No,NaN,10.1007/BF01475382,0028-1042
42911,18531,Naturwissenschaft und reale Aussenwelt,(PHY) Physics;(SOC) Philosophy;,Berlin,Die Naturwissenschaften,Springer,Germany,Max Planck,NaN,Review Article;,...,0.0,1940-12-01 00:00:00,10.1007/BF01488952,0.0,Retraction,+Copyright Claims;,No,exact date of retraction unknown,10.1007/BF01488952,0028-1042
42912,17246,The Trend-Seasonal Normal in Time Series,(ENV) Climatology;,Yale University,Journal of the American Statistical Association,Taylor and Francis,United States,C H Whelden Jr,http://retractionwatch.com/2014/07/14/dipping-...,Review Article;,...,0.0,1926-04-01 00:00:00,unavailable,0.0,Retraction,+Error in Analyses;,No,NaN,NaN,NaN


In [646]:
rw.shape

(42700, 20)

In [647]:
rw_w_issns.drop_duplicates(inplace=True)
rw_w_issns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42704 entries, 0 to 42913
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              42704 non-null  int64  
 1   Title                  42704 non-null  object 
 2   Subject                42704 non-null  object 
 3   Institution            42703 non-null  object 
 4   Journal                42704 non-null  object 
 5   Publisher              42704 non-null  object 
 6   Country                42704 non-null  object 
 7   Author                 42704 non-null  object 
 8   URLS                   21687 non-null  object 
 9   ArticleType            42704 non-null  object 
 10  RetractionDate         42704 non-null  object 
 11  RetractionDOI          42213 non-null  object 
 12  RetractionPubMedID     37603 non-null  float64
 13  OriginalPaperDate      42704 non-null  object 
 14  OriginalPaperDOI       40177 non-null  object 
 15  Origina

In [648]:
rw_w_issns[rw_w_issns.duplicated(subset='Record ID', keep=False)]

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,...,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes,DOI,ISSN
596,46438,Melatonin attenuates dysregulation of the circ...,(BLS) Biochemistry;(BLS) Biology - Cellular;,"Institute of Biomedicine (IBIOMED), University...",Frontiers in Pharmacology,Frontiers,Spain,BÃ¡rbara GonzÃ¡lez-FernÃ¡ndez;Diana I SÃ¡nchez...,NaN,Research Article;,...,37484010.0,2018-05-28 00:00:00,10.3389/fphar.2018.00556,29892224.0,Retraction,+Duplication of Image;+Investigation by Journa...,No,NaN,10.3389/fphar.2018.00556,NaN
597,46438,Melatonin attenuates dysregulation of the circ...,(BLS) Biochemistry;(BLS) Biology - Cellular;,"Institute of Biomedicine (IBIOMED), University...",Frontiers in Pharmacology,Frontiers,Spain,BÃ¡rbara GonzÃ¡lez-FernÃ¡ndez;Diana I SÃ¡nchez...,NaN,Research Article;,...,37484010.0,2018-05-28 00:00:00,10.3389/fphar.2018.00556,29892224.0,Retraction,+Duplication of Image;+Investigation by Journa...,No,NaN,10.3389/fphar.2018.00556,1663-9812
7145,36821,Extra-axial chordoid meningioma: a case report...,(HSC) Medicine - Neurology;(HSC) Medicine - On...,"Service de Neurochirurgie, Centre Hospitalier ...",Pan African Medical Journal (PAMJ),Pan African Medical Journal (PAMJ),Morocco,Mohammed Guini;Mohammed Khoulali;Nabil Raouzi;...,NaN,Case Report;,...,35480406.0,2021-02-04 00:00:00,10.11604/pamj.2021.38.123.20437,33912293.0,Retraction,+False/Forged Authorship;,No,NaN,10.11604/pamj.2021.38.123.20437,NaN
7146,36821,Extra-axial chordoid meningioma: a case report...,(HSC) Medicine - Neurology;(HSC) Medicine - On...,"Service de Neurochirurgie, Centre Hospitalier ...",Pan African Medical Journal (PAMJ),Pan African Medical Journal (PAMJ),Morocco,Mohammed Guini;Mohammed Khoulali;Nabil Raouzi;...,NaN,Case Report;,...,35480406.0,2021-02-04 00:00:00,10.11604/pamj.2021.38.123.20437,33912293.0,Retraction,+False/Forged Authorship;,No,NaN,10.11604/pamj.2021.38.123.20437,1937-8688
23999,43603,"AvaliaÃ§Ã£o bioquÃ­mica, hormonal e genÃ©tica ...",(BLS) Genetics;(HSC) Medicine - Endocrinology;,Instituto Estadual de Diabetes e Endocrinologi...,Arquivos Brasileiros de Endocrinologia & Metab...,Sociedade Brasileira de Endocrinologia e Metab...,Brazil,Dayse Caldas;Wellington Santana da Silva Jr;Jo...,NaN,Case Report;,...,27191051.0,2013-12-11 00:00:00,10.1590/S0004-27302013000800002,24343626.0,Retraction,+Concerns/Issues About Image;+Informed/Patient...,No,NaN,10.1590/S0004-27302013000800002,2359-3997
24000,43603,"AvaliaÃ§Ã£o bioquÃ­mica, hormonal e genÃ©tica ...",(BLS) Genetics;(HSC) Medicine - Endocrinology;,Instituto Estadual de Diabetes e Endocrinologi...,Arquivos Brasileiros de Endocrinologia & Metab...,Sociedade Brasileira de Endocrinologia e Metab...,Brazil,Dayse Caldas;Wellington Santana da Silva Jr;Jo...,NaN,Case Report;,...,27191051.0,2013-12-11 00:00:00,10.1590/S0004-27302013000800002,24343626.0,Retraction,+Concerns/Issues About Image;+Informed/Patient...,No,NaN,10.1590/S0004-27302013000800002,0004-2730
41370,6804,Cardiac Troponin T: An Early Molecule Marker o...,(HSC) Medicine - Cardiology;(HSC) Medicine - O...,"Department of Obstetrics and Gynecology, Renmi...",Cardiology,Karger,China,Yan-Bo Li;Cheng-Lin Hu;Jia-Ming Zhang;Yong-Gua...,NaN,Research Article;,...,0.0,2007-02-13 00:00:00,10.1159/000099107,17299263.0,Retraction,+Duplication of Article;+Euphemisms for Duplic...,No,NaN,10.1159/000099107,1355-6037
41371,6804,Cardiac Troponin T: An Early Molecule Marker o...,(HSC) Medicine - Cardiology;(HSC) Medicine - O...,"Department of Obstetrics and Gynecology, Renmi...",Cardiology,Karger,China,Yan-Bo Li;Cheng-Lin Hu;Jia-Ming Zhang;Yong-Gua...,NaN,Research Article;,...,0.0,2007-02-13 00:00:00,10.1159/000099107,17299263.0,Retraction,+Duplication of Article;+Euphemisms for Duplic...,No,NaN,10.1159/000099107,0008-6312


In [649]:
# Identify duplicates and mark the last occurrence for removal
duplicates_to_drop = rw_w_issns.duplicated(subset='Record ID', keep='last')

# Filter the DataFrame to keep only the rows where the last occurrence of duplicates is True
rw_w_issns = rw_w_issns[~duplicates_to_drop]

In [650]:
rw_w_issns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42700 entries, 0 to 42913
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              42700 non-null  int64  
 1   Title                  42700 non-null  object 
 2   Subject                42700 non-null  object 
 3   Institution            42699 non-null  object 
 4   Journal                42700 non-null  object 
 5   Publisher              42700 non-null  object 
 6   Country                42700 non-null  object 
 7   Author                 42700 non-null  object 
 8   URLS                   21687 non-null  object 
 9   ArticleType            42700 non-null  object 
 10  RetractionDate         42700 non-null  object 
 11  RetractionDOI          42209 non-null  object 
 12  RetractionPubMedID     37599 non-null  float64
 13  OriginalPaperDate      42700 non-null  object 
 14  OriginalPaperDOI       40173 non-null  object 
 15  Origina

In [651]:
rw_w_issns['ISSN'] = rw_w_issns.groupby('Journal')['ISSN'].fillna(method='ffill')

C:\Users\isabe\AppData\Local\Temp\ipykernel_10760\2693084175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rw_w_issns['ISSN'] = rw_w_issns.groupby('Journal')['ISSN'].fillna(method='ffill')


In [652]:
rw_w_issns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42700 entries, 0 to 42913
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              42700 non-null  int64  
 1   Title                  42700 non-null  object 
 2   Subject                42700 non-null  object 
 3   Institution            42699 non-null  object 
 4   Journal                42700 non-null  object 
 5   Publisher              42700 non-null  object 
 6   Country                42700 non-null  object 
 7   Author                 42700 non-null  object 
 8   URLS                   21687 non-null  object 
 9   ArticleType            42700 non-null  object 
 10  RetractionDate         42700 non-null  object 
 11  RetractionDOI          42209 non-null  object 
 12  RetractionPubMedID     37599 non-null  float64
 13  OriginalPaperDate      42700 non-null  object 
 14  OriginalPaperDOI       40173 non-null  object 
 15  Origina

In [653]:
rw_w_issns['Journal'].nunique()

7904

In [654]:
# Group by 'Journal' and check if any group only contains "unavailable" 'OriginalPaperDOI'
journals_with_only_unavailable_doi = rw_w_issns.groupby('Journal').filter(lambda x: all(x['OriginalPaperDOI'].str.lower() == 'unavailable'))

# Extract unique journal names from the result
journals_list = journals_with_only_unavailable_doi['Journal'].unique()
len(journals_list)

1054

In [679]:
rw_w_issns[rw_w_issns['ISSN'].isna()]['Journal'].unique()

array(['Science', 'Journal of Hypertension',
       'Journal of Ovarian Research', ..., 'Intermediate Algebra, 2nd ed',
       'The American Journal of Cardiology', 'Die Naturwissenschaften'],
      dtype=object)

In [689]:
len(set(doi_per_journal.index))

506

In [692]:
len(rw_w_issns[rw_w_issns['ISSN'].isna()]['Journal'].unique())-1054-2375

1940

In [691]:
len(set(doi_per_journal.index).intersection(rw_w_issns[rw_w_issns['ISSN'].isna()]['Journal'].unique()))

506

In [655]:
articles_per_journal1 = rw_w_issns[rw_w_issns['ISSN'].isna()].groupby('Journal').size()
articles_per_journal1

Journal
'Herbst des Mittelalters' Fragen zur Bewertung des 14th und 15th Jahrhunderts                                           1
... fÃ¼r unseren Betrieb lebensnotwendig ...: Georg von Holtzbrinck als Verlagsunternehmer im Dritten Reich             3
10th IEEE International Conference on Electronics, Circuits and Systems, 2003. ICECS 2003                               1
10th International Conference on Telecommunications, 2003                                                               1
12th IEEE International Conference and Workshops on the Engineering of Computer-Based Systems (ECBS'05)                 1
                                                                                                                       ..
medRxiv                                                                                                                22
medrxiv                                                                                                                 1
microPublication

In [656]:
articles_per_journal1[articles_per_journal1>1].sort_values()[-30:]

Journal
2010 2nd International Conference on Advanced Computer Control                                           72
2010 2nd IITA International Conference on Geoscience and Remote Sensing                                  82
2011 IEEE Power Engineering and Automation Conference                                                    86
2010 International Conference on Educational and Information Technology (ICEIT)                          88
2011 International Conference on Product Innovation Management (ICPIM 2011)                              91
2011 Seventh International Conference on Natural Computation                                             95
2010 2nd International Conference on Mechanical and Electronics Engineering                             107
2010 International Conference on Optics, Photonics and Energy Engineering (OPEE)                        114
PACCS 2011: 2011 3rd Pacific-Asia Conference on Circuits, Communications and System                     124
2010 International C

In [657]:
rw_w_issns[~rw_w_issns['ISSN'].isna()]['Journal'].nunique()

3892

In [660]:
rw_w_issns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42700 entries, 0 to 42913
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Record ID              42700 non-null  int64  
 1   Title                  42700 non-null  object 
 2   Subject                42700 non-null  object 
 3   Institution            42699 non-null  object 
 4   Journal                42700 non-null  object 
 5   Publisher              42700 non-null  object 
 6   Country                42700 non-null  object 
 7   Author                 42700 non-null  object 
 8   URLS                   21687 non-null  object 
 9   ArticleType            42700 non-null  object 
 10  RetractionDate         42700 non-null  object 
 11  RetractionDOI          42209 non-null  object 
 12  RetractionPubMedID     37599 non-null  float64
 13  OriginalPaperDate      42700 non-null  object 
 14  OriginalPaperDOI       40173 non-null  object 
 15  Origina